# Manutenzione Predittiva: Stima della Remaining Useful Life (RUL)

Questo notebook esplora metodi per stimare la Remaining Useful Life (RUL) di componenti o macchinari a partire da dati di degradazione temporali.

L’obiettivo è fornire una stima utile per pianificare interventi di manutenzione predittiva, riducendo fermi non programmati e ottimizzando i costi operativi.

Il notebook si articola in due fasi principali:

1. **Stima statistica preliminare** basata su parametri tipici da datasheet o modelli parametrici (es. Weibull), utilizzabile quando non è ancora disponibile un archivio di dati storici.
2. **Stima basata su similarità temporale** utilizzando tecniche di Dynamic Time Warping (DTW) per confrontare curve di degrado multivariate, applicabile una volta acquisito un archivio di curve.

Verrà mostrato un esempio completo di simulazione di dati, visualizzazione, e valutazione della RUL con entrambi gli approcci.

## Stima RUL senza archivio storico: approccio statistico e basato su datasheet

Quando l'archivio di curve storiche non è ancora disponibile (ad esempio nelle prime fasi di monitoraggio di una macchina o di un componente), non è possibile applicare direttamente metodi di similitudine temporale come il DTW.

In questa fase iniziale, possiamo sfruttare:

- Parametri statistici derivati dai datasheet delle macchine (es. vita utile media, deviazione standard della durata),
- Modelli probabilistici semplici (es. distribuzioni di Weibull, esponenziale) che descrivono la durata attesa del componente,
- Indicatori di degrado basati su soglie di segnali fisici o di processo.

Questo approccio fornisce una stima grossolana della Remaining Useful Life (RUL), che può essere raffinata in seguito quando l’archivio dati sarà sufficientemente popolato.

In questo notebook, mostreremo un esempio semplice di stima statistica basata su una distribuzione parametrica tipica (Weibull), usando parametri ipotetici ricavati da datasheet.

### Stima della RUL con distribuzione di Weibull

La distribuzione di Weibull è un modello statistico ampiamente utilizzato per rappresentare i tempi di guasto o vita utile di componenti industriali. 

Si caratterizza principalmente per il **parametro di forma** (shape), che descrive il comportamento del guasto nel tempo:
- se è < 1, il tasso di guasto diminuisce (guasti precoci);
- se è = 1, il guasto avviene con tasso costante (guasti casuali);
- se è > 1, il tasso di guasto aumenta nel tempo (usura).

Il **parametro di scala** (scale) definisce la durata tipica del componente.

Con la Weibull possiamo stimare la probabilità che un componente sopravviva fino a un certo tempo e quindi calcolare la **Remaining Useful Life (RUL)** come tempo residuo medio di vita, condizionata al fatto che il componente ha già funzionato per un certo intervallo.

Questo approccio è particolarmente utile quando non si dispone ancora di dati storici reali, ma si hanno informazioni statistiche o datasheet del produttore.


In [ ]:
from scipy.stats import weibull_min

def estimate_rul_statistical(current_time, shape=1.5, scale=1000):
    """
    Stima RUL basata su distribuzione di Weibull.
    shape: parametro di forma (beta)
    scale: parametro di scala (eta), ovvero vita media attesa
    current_time: tempo attuale di funzionamento
    
    Restituisce la vita residua media stimata a partire da current_time.
    """
    if current_time >= scale:
        return 0
    
    survival_prob = weibull_min.sf(current_time, c=shape, scale=scale)
    expected_life_total = weibull_min.mean(c=shape, scale=scale)
    
    expected_remaining_life = expected_life_total * survival_prob
    return max(0, expected_remaining_life - current_time)

# Esempio uso con current_time scelto
current_time_example = 150
estimated_rul_stat = estimate_rul_statistical(current_time_example)

print(f"Tempo attuale di funzionamento: {current_time_example}")
print(f"RUL stimata (approccio statistico): {estimated_rul_stat:.2f} unità di tempo")

### Utilizzo della stima statistica

Questa stima fornisce una baseline RUL nelle fasi iniziali senza dati storici. Quando si inizieranno a raccogliere dati reali di funzionamento (curve di degradazione), sarà possibile passare a metodi più sofisticati basati su similarità dinamica.

La combinazione di queste due fasi consente una strategia di manutenzione predittiva che parte da ipotesi generali e si adatta progressivamente all’esperienza reale del macchinario.

## Stima della RUL basata su similarità temporale (DTW)

Una volta disponibile un archivio di curve storiche di degrado, è possibile stimare la RUL del componente corrente confrontandone la traiettoria di degradazione parziale con quelle archiviate.

Questo approccio si basa sul Dynamic Time Warping (DTW), una tecnica per misurare la somiglianza tra sequenze temporali anche se di lunghezza diversa o con andamento non lineare.

Nel notebook implementiamo un algoritmo che, data una curva parziale corrente, individua la curva più simile nell'archivio e usa la differenza di lunghezza per stimare la RUL residua.

Questo metodo è particolarmente utile per sistemi multivariati dove più feature sono monitorate contemporaneamente.

### Simulazione e visualizzazione delle curve di degrado

Per testare gli algoritmi, generiamo un archivio sintetico di curve multivariate di degrado simulato. Ogni curva rappresenta un profilo di feature di degradazione nel tempo, con un'esplosione esponenziale verso la fine.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from fastdtw import fastdtw


def generate_archive(num_curves=100, min_length=100, max_length=1000, n_features=3):
    archive = []
    for _ in range(num_curves):
        length = np.random.randint(min_length, max_length + 1)
        features = simulate_curve(length=length, n_features=n_features)
        archive.append(features)
    return archive


def plot_archive(archive):
    n_features = archive[0].shape[1]

    _, axs = plt.subplots(n_features, 1, figsize=(10, 3 * n_features), sharex=True)
    if n_features == 1:
        axs = [axs]

    for i in range(n_features):
        for curve in archive:
            axs[i].plot(curve[:, i], alpha=0.7)
        axs[i].set_title(f"Feature {i + 1}")
        axs[i].set_ylabel("Value")
    axs[-1].set_xlabel("Time")
    plt.tight_layout()
    plt.show()


def simulate_curve(length=500, n_features=3, explosion_start_ratio=0.8):
    t = np.linspace(0, 1, length)
    explosion_start = explosion_start_ratio

    features = []
    for _ in range(n_features):
        noise = 0.02 * np.random.randn(length)

        explosion = np.exp(t - explosion_start)
        explosion = explosion - explosion[int(explosion_start * length)]
        explosion = np.clip(explosion, 0, None)

        c, factor = np.random.uniform(0.1, 2), np.random.uniform(5, 15)
        feature_curve = np.exp(c * t + factor * explosion) + noise
        features.append(feature_curve)

    features = np.vstack(features).T
    return features


np.random.seed(42)

archive = generate_archive(100)
plot_archive(archive)

### Test della stima RUL basata su archivio

Simuliamo una curva parziale corrente di lunghezza casuale e stimiamo la RUL usando la similarità con le curve dell'archivio.

In [ ]:
current_time = np.random.randint(200, 400)
partial_curve = simulate_curve()[:][:current_time]

In [ ]:
def estimate_rul_by_similarity(archive, partial_curve):
    partial_len = len(partial_curve)
    best_dist = np.inf
    best_idx = -1
    for idx, curve in enumerate(archive):
        if len(curve) < partial_len:
            continue
        dist, _ = fastdtw(
            partial_curve[:partial_len], curve[:partial_len], dist=euclidean_dist
        )
        if dist < best_dist:
            best_dist = dist
            best_idx = idx
    estimated_rul = len(archive[best_idx]) - partial_len
    return estimated_rul, archive[best_idx], best_idx


def euclidean_dist(a, b):
    return np.linalg.norm(a - b)


estimated_rul, best_curve, best_idx = estimate_rul_by_similarity(archive, partial_curve)

print(f"Indice curva più simile: {best_idx}")
print(f"RUL stimata al timestep {len(partial_curve)}: {estimated_rul:.2f}")

In [ ]:
n_features = partial_curve.shape[1]

plt.figure(figsize=(12, 3 * n_features))
for i in range(n_features):
    plt.subplot(n_features, 1, i + 1)
    plt.title(f"Feature {i} - Curva corrente vs Curva più simile")
    plt.plot(partial_curve[:, i], "-o", label="Curva corrente")
    plt.plot(best_curve[:, i], label="Curva più simile")
    plt.legend()
plt.tight_layout()
plt.show()

## Conclusioni

In questo notebook abbiamo esplorato due approcci complementari per la stima della Remaining Useful Life (RUL):

- Un metodo statistico preliminare basato su modelli parametrici (Weibull) utile quando non è ancora disponibile un archivio storico,
- Un metodo basato su similarità temporale multivariata (DTW) applicato a un archivio di curve di degradazione raccolte nel tempo.

Questo workflow permette di affrontare la manutenzione predittiva in maniera graduale e flessibile, passando da stime basate su dati teorici e specifiche di macchina, a stime basate su dati operativi effettivi.

L’approccio proposto può essere ulteriormente arricchito integrando modelli di machine learning più avanzati, sensori aggiuntivi e meccanismi di aggiornamento continuo dei modelli di RUL man mano che i dati vengono acquisiti.